In [16]:
from pathlib import Path
from tqdm import tqdm

import math
import numpy as np
import pandas as pd
import os
import json

In [2]:
PATH = Path('data')
data = None
with open(PATH/'youcookii_annotations_trainval.json', 'rb') as f:
    data = json.load(f)

In [5]:
text_data_file = PATH/'youcook_text.csv'
text_dataset = []

for _id, rec in tqdm(data['database'].items()):
    recipe_type = rec['recipe_type']
    subset = rec['subset']
    for ann in rec['annotations']:
        text_dataset.append([recipe_type, _id, subset, ann['id'], ann['segment'], ann['sentence']])
df = pd.DataFrame(text_dataset, columns=['recipe_type', 'key', 'subset', 'id', 'segment', 'sentence'])
df.to_csv(text_data_file, index=False)

100%|████████████████████████████████████████████████████████████████████| 1790/1790 [00:00<00:00, 256703.39it/s]


In [43]:
data['database']['GLd3aX16zBg']

{'duration': 241.62,
 'subset': 'training',
 'recipe_type': '113',
 'annotations': [{'segment': [90, 102],
   'id': 0,
   'sentence': 'spread margarine on two slices of white bread'},
  {'segment': [114, 127],
   'id': 1,
   'sentence': 'place a slice of cheese on the bread'},
  {'segment': [132, 138],
   'id': 2,
   'sentence': 'place the bread slices on top of each other and place in a hot pan'},
  {'segment': [139, 145],
   'id': 3,
   'sentence': 'flip the sandwich over and press down'},
  {'segment': [173, 174],
   'id': 4,
   'sentence': 'cut the sandwich in half diagonally'}],
 'video_url': 'https://www.youtube.com/watch?v=GLd3aX16zBg'}

In [8]:
text_data_file = PATH/'youcook_text.csv'
df = pd.read_csv(text_data_file)
df.head(100)

,recipe_type,key,subset,id,segment,sentence
0,113,GLd3aX16zBg,training,0,"[90, 102]",spread margarine on two slices of white bread
1,113,GLd3aX16zBg,training,1,"[114, 127]",place a slice of cheese on the bread
2,113,GLd3aX16zBg,training,2,"[132, 138]",place the bread slices on top of each other an...
3,113,GLd3aX16zBg,training,3,"[139, 145]",flip the sandwich over and press down
4,113,GLd3aX16zBg,training,4,"[173, 174]",cut the sandwich in half diagonally
...,...,...,...,...,...,...
95,423,4AIahLZfNq4,training,1,"[84, 110]",add shrimp paste cloves tumeric and curry powd...
96,423,4AIahLZfNq4,training,2,"[141, 190]",add oil to the food processor while blending
97,423,4AIahLZfNq4,training,3,"[200, 228]",blend the ingredients in the food processor
98,423,4AIahLZfNq4,training,4,"[274, 286]",stir the curry paste


In [ ]:
pd.read_csv

In [ ]:
sorted(os.listdir(PATH/'feat_dat/train_frame_feat/102/0IuQKThr-pM/0001'))

np.loadtxt(PATH/'feat_dat/train_frame_feat/102/0IuQKThr-pM/0001/resnet_34_feat_mscoco.dat')

In [ ]:
sorted(os.listdir(PATH/'features'/'feat_csv/train_frame_feat_csv/101'))

In [ ]:
df = pd.read_csv(PATH/'features'/'feat_csv/train_frame_feat_csv/113/2vNPfc8LaTc/0010/resnet_34_feat_mscoco.csv', header=None)
df.shape

In [ ]:
# train_split = None
# with open(PATH/'splits/train_list.txt', 'r') as f:
#     train_split = f.readlines()
# train_split
os.listdir(PATH/'splits')

In [72]:
df_linker = pd.read_csv(PATH/'splits'/'train_duration_totalframe.csv')
# df_linker[df_linker['vid_id'] == '2vNPfc8LaTc']
df_linker['I'] = (df_linker['total_frame']/500).apply(math.ceil)
df_linker['fps'] = (df_linker['total_frame']/df_linker['duration']).apply(math.ceil)
df_linker['spf'] = (1/df_linker['fps'])
df_linker.head()

,vid_id,duration,total_frame,I,fps,spf
0,-xbTvALWCIg,455.02,10908,22,24,0.041667
1,8X2wDyjl6T8,46.00,1377,3,30,0.033333
2,bczCfVC6HYU,201.42,5035,11,25,0.040000
3,aMtZQgH_MRc,163.02,4889,10,30,0.033333
4,RV0_DWkbjx8,135.88,4072,9,30,0.033333


In [66]:
# df.set_index('key').join(df_linker.set_index('vid_id')).head(20)

In [73]:
# df_linker['frames_needed'] = [None] * df_linker.shape[0]
frames_disected = []
for idx, row in df_linker.iterrows():
    frames_needed = []
    frames_repre = []
    m = 0
    while m < row['total_frame']:
        frames_needed.append(m)
        frames_repre.append(m * row['spf'])
        m += row['I']
#     print(frames_needed)
    frames_disected.append([row['vid_id'], frames_needed.copy(), frames_repre.copy()])
#     break
# df_linker.head()
# df_linker['frames_needed'] = df_linker['fps']/df_linker['I']

In [104]:
df_segments = pd.DataFrame(frames_disected, columns=['vid_id', 'frames_recorded', 'segments']).set_index('vid_id').join(df.set_index('key'))
df_segments['segment'] = df_segments['segment'].apply(eval)
df_segments.head()

,frames_recorded,segments,recipe_type,subset,id,segment,sentence
--bv0V6ZjWI,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[0.0, 0.6666666666666666, 1.3333333333333333, ...",205,training,0,"[8, 23]",crush and chop the garlic
--bv0V6ZjWI,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[0.0, 0.6666666666666666, 1.3333333333333333, ...",205,training,1,"[23, 47]",add oil garlic and salt to a bowl
--bv0V6ZjWI,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[0.0, 0.6666666666666666, 1.3333333333333333, ...",205,training,2,"[64, 76]",mix the tomoatoes with the oil mixture
--bv0V6ZjWI,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[0.0, 0.6666666666666666, 1.3333333333333333, ...",205,training,3,"[81, 90]",chop the basil
--bv0V6ZjWI,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[0.0, 0.6666666666666666, 1.3333333333333333, ...",205,training,4,"[90, 116]",spread the oil mixture onto the dough


In [115]:
frames_needed_for_extract = []
for idx, row in df_segments.iterrows():
    start, end = tuple(row['segment'])
    arr = np.array(row['segments'])
    arg_index = np.argwhere((arr >= start) & (arr < end))
    frames_needed_for_extract.append([idx, arg_index.flatten().tolist()])

df_extract = pd.DataFrame(frames_needed_for_extract, columns=['vid_id', 'extract'])

In [120]:
df.set_index('key').join(df_extract.set_index('vid_id'))

,recipe_type,subset,id,segment,sentence,extract
--bv0V6ZjWI,205,training,0,"[8, 23]",crush and chop the garlic,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2..."
--bv0V6ZjWI,205,training,0,"[8, 23]",crush and chop the garlic,"[35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 4..."
--bv0V6ZjWI,205,training,0,"[8, 23]",crush and chop the garlic,"[96, 97, 98, 99, 100, 101, 102, 103, 104, 105,..."
--bv0V6ZjWI,205,training,0,"[8, 23]",crush and chop the garlic,"[122, 123, 124, 125, 126, 127, 128, 129, 130, ..."
--bv0V6ZjWI,205,training,0,"[8, 23]",crush and chop the garlic,"[135, 136, 137, 138, 139, 140, 141, 142, 143, ..."
...,...,...,...,...,...,...
zzT6RoI4JPU,221,validation,8,"[472, 514]",pour some warm water to it and cook until the ...,NaN
zzT6RoI4JPU,221,validation,9,"[523, 565]",melt some better in another small pan and add ...,NaN
zzT6RoI4JPU,221,validation,10,"[566, 621]",add the roasted flour to the cooking meat mixt...,NaN
zzT6RoI4JPU,221,validation,11,"[622, 638]",finely chop marjoram and add it to the goulash,NaN


In [33]:
all_df = df.set_index('key').join(df_linker.set_index('vid_id'))

In [35]:
all_df = all_df[all_df['subset'] == 'training']


--bv0V6ZjWI    [training]
--bv0V6ZjWI    [training]
--bv0V6ZjWI    [training]
--bv0V6ZjWI    [training]
--bv0V6ZjWI    [training]
                  ...    
zwbCD3CUFIs    [training]
zwbCD3CUFIs    [training]
zwbCD3CUFIs    [training]
zwbCD3CUFIs    [training]
zwbCD3CUFIs    [training]
Name: subset, Length: 10337, dtype: object

In [42]:
all_df['segment']

dtype('O')